# Assignment 5

### Use this notebook to complete the Assignment 5
### This gives a base program to complete modules in Assignment 5

In [ ]:
!pip install pyspark

In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext, DataFrame
from pyspark.sql.functions import desc,monotonically_increasing_id,sum,lit,col

import os
import re
from functools import reduce

#### This program works only for Spark 3.0 and Scala 2.12

In [ ]:
SUBMIT_ARGS = "--packages graphframes:graphframes:0.8.0-spark3.0-s_2.12 pyspark-shell"
os.environ["PYSPARK_SUBMIT_ARGS"] = SUBMIT_ARGS

In [ ]:
conf = SparkConf().setAppName('GraphFrames').setMaster('local')
sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)

In [ ]:
from graphframes import *

In [ ]:
# Useful functions to explore before this assignment
# 1. Dataframe's select
# 2. Dataframe's groupBy
# 3. Dataframe's df.schema.names
# 4. Dataframe's df.printSchema()
# 5. Dataframe's take()

### Module - 1: Data cleaning and processing
### Steps:
#### 1. Use wiki_tiny.txt as data for this Assignment
#### 2. Create dataRDD from the input file
#### 3. The data is very similar to Assignment-3. But this time it is structured as <title>Doc title</title><text>Doc content</text>, where the Doc content contains many URLs as [[URL]]
#### 4. Write a regular Spark program to clean the Doc content and get all edges represented as (Doc title, url) 

In [ ]:
# Give data path here
data_path = 'wiki_tiny.txt'

In [ ]:
dataRDD = sc.textFile(data_path) 

In [ ]:
# Function to clean the text 
# 1. Get title and text of each document. URLs in documents are represented by [[URL]] and it should be retrieved using regular expressions
# 2. URLs in each document may have special cases. Example: [[source:background.jpeg|Background]]. 
#                                                  For such cases consider only "source:background.jpeg" as URL
# 3. The function should return a list of tuples (title,URL) - Figure out how to achieve that
def get_edges(line):
    to_return = []

    # Complete the data processing steps here

    return to_return
# Figure out which Spark transformation function to use here
# From a document (wikipedia page), in the given file, we need a list of (title,URL)
# Replace "transformation" from the following line to the corresponding function name. Example replace it with "map"
edgesRDD = dataRDD.transformation(get_edges)

### Module - 2: Graph creation 
### Steps:
#### 1. Create edges dataframe
#### 2. Create vertices dataframe from edges
#### 3. Create graph dataframe

In [ ]:
# TASK 2.1
# Convert edgesRDD to a dataframe (edgesDF)
# The edgesRDD columns should be named "src" and "dst"
# HINT: Use toDF()
edgesDF = 

In [ ]:
# TASK 2.2
# Get dataframes for source and destination vertices from edgesDF's src and dst columns respectively
# Each of these dataframes should only have unique values - no duplicates - as we need a unique list of vertex names to create 'id'
# Complete the following statements
srcDF = edgesDF.
destDF = edgesDF.

In [ ]:
# TASK 2.3
# Union vertices from srcDF and destDF, which will give back another dataframe (named verticesDF) with only one column
# HINT: Use python functools's reduce function 

# Complete the following statement to union
verticesDF = 

# Rename the column as "Page" and assign back to the same variable (verticesDF)
verticesDF = verticesDF.

# Add an ID column to the dataframe and assign back to the same variable (verticesDF)
# HINT: Use monotonically_increasing_id()
verticesDF = verticesDF.

In [ ]:
# Create a graphframe from verticesDF and edgesDF
G = GraphFrame(verticesDF,edgesDF)

### Module - 3: Basic queries on the constructed graph

In [ ]:
# TASK 3.1
# Write a query to show number of vertices in the graph


In [ ]:
# TASK 3.2
# Write a query to show number of edges in the graph


In [ ]:
# TASK 3.3
# 1. Get indegrees of the graph
# 2. Sort them in descending order 
# 3. Display only top 15 rows of the dataframe
# The output will display which page in wikipedia is having more in-links


In [ ]:
# TASK 3.4
# 1. Get degrees of the graph
# 2. Sort them in descending order 
# 3. Display only top 15 rows of the dataframe
# The output will display which page in wikipedia is having more links(sum of in-links and out-links)


In [ ]:
# TASK 3.5
# 1. Get number of multiple edges between nodes in graph G
# The graph created from Wikipedia articles have multiple edges. Example: 150 edges from the node "April" to the node "August"
# This means there the page "April" has reference the page "August" 150 times
# 2. The resulting dataframe should have only three columns: "src", "dst", and "count", where count is number of edges between "src" and "dst"
# 3. Sort the dataframe is descending order
# 4. Display only top 15 rows of the dataframe
# HINT: This can be achieved with a sequence of very simple dataframe commands 


### Module - 4: Message Aggregation - Page Ranking

In [ ]:
# 1. Get the graph G's outdegree to outDegDF
outDegDF = 

# 2. Join outDegDF and verticesDF. Assign the joined dataframes to verticesDF
# 3. Finally, verticesDF should have only 3 columns: id, Page, and outDegree 
verticesDF = 

# 4. Create a new graph new_G with new verticesDF and existing edgesDF
new_G = GraphFrame(verticesDF,edgesDF)

In [ ]:
# 1. Write an aggregateMessages function on 'new_G' to sum neighbors' outDegree for each vertex 
# [NOTE: messages should pass only via the edge direction, not in both ways as discussed in the lecture]
# 2. The aggregated outDegree will be the rank of a page (higher the rank of a page, higher its importance )
# 3. Show top 15 very important pages along with their rank